In [7]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/SushankMishra/DataScienceProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "SushankMishra"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9c8d71ade4339f2f6ae056382c9da10c282d95f8"

In [8]:
os.chdir('/Users/sushankmishra/Desktop/untitled folder/Newp/DataScienceProject')
print(os.getcwd()) 

/Users/sushankmishra/Desktop/untitled folder/Newp/DataScienceProject


In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metrics_file_name: Path
    target_column: str
    mlflow_uri: str

In [10]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH, schema_file_path = SCHEMA_FILE_PATH):
        self.config = config_file_path
        self.params = params_file_path
        self.schema = schema_file_path
        self.config = read_yaml(self.config)
        self.params = read_yaml(self.params)
        self.schema = read_yaml(self.schema)
    
        create_directories([self.config.artifacts_root])
    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = self.params.ElasticNet,
            metrics_file_name = config.metrics_file_name,
            target_column = self.schema.TARGET_COLUMN,
            mlflow_uri = os.environ.get("MLFLOW_TRACKING_URI")
        )
        return model_evaluation_config

In [11]:
from urllib.parse import urlparse
from sklearn.metrics import r2_score
from src.datascience.utils.common import save_json
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
import numpy as np
import pandas as pd
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(np.mean((pred - actual) ** 2))
        mae = np.mean(np.abs(pred - actual))
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        X = test_data.drop(self.config.target_column.name, axis=1)
        y = test_data[self.config.target_column.name]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(self.config.mlflow_uri).scheme
        
        mlflow.set_experiment("model_evaluation_experiment")    
        with mlflow.start_run():
            y_pred = model.predict(X)
            rmse, mae, r2 = self.eval_metrics(y, y_pred)
            save_json(path = Path(self.config.metrics_file_name), data={
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            })
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            mlflow.log_params(self.config.all_params)
            signature = infer_signature(X, y)
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", signature=signature, registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

2025-08-02 22:44:52,513 - INFO - YAML file read successfully from config/config.yaml
2025-08-02 22:44:52,517 - INFO - YAML file read successfully from params.yaml
2025-08-02 22:44:52,519 - INFO - YAML file read successfully from schema.yaml
2025-08-02 22:44:52,520 - INFO - Directory created: artifacts
2025-08-02 22:44:52,520 - INFO - Directory created: artifacts/model_evaluation
2025-08-02 22:44:53,399 - INFO - JSON file saved at artifacts/model_evaluation/metrics.json


/Users/sushankmishra/Desktop/untitled folder/Newp/DataScienceProject/venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'ElasticNetModel'.
2025/08/02 22:45:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Mod

🏃 View run gaudy-hare-72 at: https://dagshub.com/SushankMishra/DataScienceProject.mlflow/#/experiments/0/runs/06003900bbd547338931697973051886
🧪 View experiment at: https://dagshub.com/SushankMishra/DataScienceProject.mlflow/#/experiments/0
